# <center>Basic Model for Seasonal Inventory Prebuild using Optimization</center>

<center>Sabi Horvat, January 2021</center>

Manufacturing facilities may not have the capacity to produce seasonal products within that season. Even if the facilities can surge capacity to meet demand, that may result in additional overhead costs, including nonlinear overtime pay.  The cost of storing prebuilt product in inventory may be less than this added cost.  

When taking the prebuild approach, there's a few questions to answer:
* What is the quantity of the product to prebuild into inventory before the season starts?
* Assuming the prebuilt inventory will build up over time with a smoothed production plan, when does manufacturing need to start for the prebuild? 

Although, the simplified model could be worked out with pen and paper, this model provides the scalable framework to build upon for multiple products amid competing capacity constraints.  

And here's additional python formulations of multi-period production schedules that you might find helpful: 
* [Ehsan Khodabandeh's tutorial on Medium](https://medium.com/@ehsankhoda/tutorial-a-simple-framework-for-optimization-programming-in-python-using-pulp-and-gurobi-1e73e76532f2)
* [Ben Keen on Github](https://github.com/benalexkeen/Introduction-to-linear-programming/blob/master/Introduction%20to%20Linear%20Programming%20with%20Python%20-%20Part%205.ipynb)

---
## Python Model

In [1]:
import pandas as pd
from pulp import *

In [2]:
# data for one product
Max_Capacity = { 1: 20000, 2: 20000, 3: 20000, 4: 20000 } # assuming stable capacity
Demand = {1: 10000, 2: 12000, 3: 25000, 4: 22000,}
Variable_Cost = {1:10, 2:10, 3:10, 4:10} # not accounting for overtime with capacity increase in this simplified model
Months = [1,2,3,4]
Initial_Inventory = 1000

In [3]:
# create model
model = LpProblem(name="scheduling_problem", sense=LpMinimize)

# decision variables
x = LpVariable.dicts("produced",Months,lowBound=0, cat='integer')

# other variables
b = LpVariable.dicts("beginning_inventory",Months,lowBound=0, cat='integer')
e = LpVariable.dicts("ending_inventory",Months,lowBound=0, cat='integer')

# Add the objective function to the model
model += lpSum(Variable_Cost[m]*x[m] for m in Months)

# Add the constraints to the model

# initial month constraints
model += b[1] == Initial_Inventory, "Beginning_inventory_initial"
model += Initial_Inventory + x[1] >= Demand[1] + e[1], "Demand_initial: " 
model += x[1] <= Max_Capacity[1], "Capacity_initial: " 

# constraints for months after the initial month
for m in Months[1:]:
    model += b[m] == e[m-1]
    model += b[m] + x[m] >= Demand[m] + e[m], "Demand: " + str(m)
    model += x[m] <= Max_Capacity[m], "Capacity: " + str(m)

# production smoothing, lessens production changes from one period to the next 
for m in Months[1:]:
    model += x[m] >= 0.8*x[m-1]
    model += x[m-1] >= 0.8*x[m]
    
# Solve the problem
status = model.solve()

In [4]:
# the "produced" variable is likely the most interesting outcome of this model
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 680000.0
beginning_inventory_1: 1000.0
beginning_inventory_2: 11000.0
beginning_inventory_3: 15000.0
beginning_inventory_4: 4222.2222
ending_inventory_1: 11000.0
ending_inventory_2: 15000.0
ending_inventory_3: 4222.2222
ending_inventory_4: 0.0
produced_1: 20000.0
produced_2: 16000.0
produced_3: 14222.222
produced_4: 17777.778
